In [10]:
#EDA for EAR Derivation

In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.feature_selection import VarianceThreshold
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc
from tqdm.auto import tqdm
tqdm.pandas()
warnings.filterwarnings('ignore')

source data data column definations

In [12]:
column_info = pd.read_csv('HomeCredit_columns_description.csv', encoding = 'unicode_escape')

In [13]:
column_info[column_info['Table'] == 'previous_application.csv'].head(5)

,Unnamed: 0,Table,Row,Description,Special
173,176,previous_application.csv,SK_ID_PREV,ID of previous credit in Home credit related t...,hashed
174,177,previous_application.csv,SK_ID_CURR,ID of loan in our sample,hashed
175,178,previous_application.csv,NAME_CONTRACT_TYPE,"Contract product type (Cash loan, consumer loa...",NaN
176,179,previous_application.csv,AMT_ANNUITY,Annuity of previous application,NaN
177,180,previous_application.csv,AMT_APPLICATION,For how much credit did client ask on the prev...,NaN


data loading

In [53]:
payments_file = 'installments_payments.csv'
previous_file = 'previous_application.csv'
df_payments = pd.read_csv(payments_file)
df_previous = pd.read_csv(previous_file)
(df_previous.shape, df_payments.shape)

((1670214, 37), (13605401, 8))

expected EAR

In [54]:
# select only required columns for previous applications and clean up unwanted dataframes
df_previous_filtered = df_previous[['SK_ID_CURR','SK_ID_PREV','AMT_ANNUITY','AMT_CREDIT','CNT_PAYMENT']]\
[df_previous['SK_ID_CURR'] == 100042]
df_previous_filtered.shape
del df_previous
gc.collect()

48

In [55]:
# handle null values
df_previous_filtered['AMT_ANNUITY'] = df_previous_filtered['AMT_ANNUITY'].fillna(0)
df_previous_filtered['AMT_CREDIT'] = df_previous_filtered['AMT_CREDIT'].fillna(0)
df_previous_filtered['CNT_PAYMENT'] = df_previous_filtered['CNT_PAYMENT'].fillna(0)

In [56]:
def get_ear(annuity, credit, no_of_payments):
    """ derives ear and returns
    """
    PMT = [-credit]
    PMT = PMT + ([annuity] * int(no_of_payments))
    irr = np.irr(PMT)
    
    return (1+irr)**12 - 1

In [57]:
df_previous_filtered['EXPECTED_EAR'] = df_previous_filtered\
.progress_apply(lambda x: get_ear(x.AMT_ANNUITY, x.AMT_CREDIT, x.CNT_PAYMENT), axis=1)

In [58]:
#df_previous_filtered[df_previous_filtered['EXPECTED_EAR'].isnull()]
df_previous_filtered['EXPECTED_EAR'] = df_previous_filtered['EXPECTED_EAR'].fillna(0)

In [59]:
# aggregate new column at the current application level.
df_expected_ear = df_previous_filtered.groupby('SK_ID_CURR')['EXPECTED_EAR']\
.agg(['mean','median','std'])\
.rename(columns={'mean':'MEAN_EXPECTED_EAR','median':'MEDIAN_EXPECTED_EAR', 'std':'STD_EXPECTED_EAR'})

In [60]:
df_expected_ear.head(6)

,MEAN_EXPECTED_EAR,MEDIAN_EXPECTED_EAR,STD_EXPECTED_EAR
SK_ID_CURR,,,
100042,0.404926,0.413872,0.222628


In [61]:
df_expected_ear['MEAN_EXPECTED_EAR'] = df_expected_ear['MEAN_EXPECTED_EAR'].fillna(0)
df_expected_ear['MEDIAN_EXPECTED_EAR'] = df_expected_ear['MEDIAN_EXPECTED_EAR'].fillna(0)
df_expected_ear['STD_EXPECTED_EAR'] = df_expected_ear['STD_EXPECTED_EAR'].fillna(0)

In [62]:
df_expected_ear.head(6)

,MEAN_EXPECTED_EAR,MEDIAN_EXPECTED_EAR,STD_EXPECTED_EAR
SK_ID_CURR,,,
100042,0.404926,0.413872,0.222628


Actual EAR

In [63]:
# select required installments 
df_payments_filtered = df_payments[['SK_ID_PREV','DAYS_INSTALMENT','DAYS_ENTRY_PAYMENT','AMT_INSTALMENT']][df_payments['SK_ID_PREV'].isin(df_previous_filtered['SK_ID_PREV'].tolist())]

In [64]:
df_payments_filtered.shape

(182, 4)

In [65]:
# merge with previous application to get AMT_CREDIT
df_previous_payments = df_payments_filtered\
.merge(df_previous_filtered[['SK_ID_CURR','SK_ID_PREV','EXPECTED_EAR','AMT_CREDIT']], on='SK_ID_PREV', how='inner')

In [66]:
df_previous_payments.shape

(182, 7)

In [67]:
# handle nulls
df_previous_payments['AMT_CREDIT'] = df_previous_payments['AMT_CREDIT'].fillna(0)
df_previous_payments['DAYS_INSTALMENT'] = df_previous_payments['DAYS_INSTALMENT'].fillna(0)
df_previous_payments['DAYS_ENTRY_PAYMENT'] = df_previous_payments['DAYS_ENTRY_PAYMENT'].fillna(0)
df_previous_payments['AMT_INSTALMENT'] = df_previous_payments['AMT_INSTALMENT'].fillna(0)

In [68]:
# derive start date
df_previous_payments = df_previous_payments\
.join((df_previous_payments.groupby('SK_ID_PREV')['DAYS_INSTALMENT']\
.agg(['min'])-30).rename(columns={'min':'START_DATE'}), on='SK_ID_PREV')

In [69]:
# derive end date
df_previous_payments = df_previous_payments\
.join(df_previous_payments.groupby('SK_ID_PREV')['DAYS_ENTRY_PAYMENT']\
.agg(['max']).rename(columns={'max':'END_DATE'}), on='SK_ID_PREV')

In [70]:
# index column based on installments to create installaments over time.
df_previous_payments['INDEX'] = df_previous_payments['DAYS_ENTRY_PAYMENT'] - df_previous_payments['START_DATE']

In [71]:
#df_previous_payments[df_previous_payments['SK_ID_PREV'] == 2604725]

In [72]:
def get_dict(df):
    keys = df['INDEX'].tolist()
    values = df['AMT_INSTALMENT'].tolist()
    dictionary = dict(zip(keys, values))
    
    return dictionary

In [73]:
# first prepare the dictionary with index and amount paid at that particular interval
df_previous_payments_t = df_previous_payments.groupby(['SK_ID_CURR','SK_ID_PREV','START_DATE','END_DATE', 'AMT_CREDIT'])\
.progress_apply(lambda x: get_dict(x))

In [74]:
df_previous_payments_t = df_previous_payments_t.reset_index()
df_previous_payments_t.columns = ['SK_ID_CURR','SK_ID_PREV','START_DATE','END_DATE','AMT_CREDIT', 'TMP_EAR_DICT']

In [78]:
df_previous_payments_t.head(5)

,SK_ID_CURR,SK_ID_PREV,START_DATE,END_DATE,AMT_CREDIT,TMP_EAR_DICT
0,100042,1125423,-2749.0,-2461.0,95431.5,"{288.0: 10469.34, 78.0: 10602.45, 43.0: 10602...."
1,100042,1643959,-1931.0,-1705.0,247275.0,"{120.0: 18619.245, 226.0: 173670.39, 179.0: 18..."
2,100042,1643964,-2337.0,-2012.0,119925.0,"{55.0: 13431.6, 204.0: 13431.6, 148.0: 13431.6..."
3,100042,1716930,-1705.0,-1371.0,315000.0,"{207.0: 20150.55, 89.0: 20150.55, 326.0: 20150..."
4,100042,2137382,-2556.0,-17.0,90000.0,"{61.0: 4500.0, 423.0: 4500.0, 792.0: 4500.0, 2..."


In [79]:
# there are a few cases where the length of the cashflow does not match with installments.
def get_cf(SK_ID_CURR, SK_ID_PREV, START_DATE, END_DATE, AMT_CREDIT, TMP_EAR_DICT):
    credit = AMT_CREDIT
    period_length = int(END_DATE - START_DATE)
    d = TMP_EAR_DICT
    if period_length > 0:
        cf = [0] * (period_length + 1)
        cf[0] = -credit
        for index, amount in d.items():
            try:
                cf[int(index)] = amount
            except Exception as e:
                print(SK_ID_CURR, SK_ID_PREV, START_DATE, END_DATE, AMT_CREDIT, TMP_EAR_DICT)
    else:
        return [0]
    return cf

In [80]:
df_previous_payments_tt = df_previous_payments_t\
.progress_apply(lambda x: get_cf(x.SK_ID_CURR, x.SK_ID_PREV, x.START_DATE, x.END_DATE, x.AMT_CREDIT, x.TMP_EAR_DICT), axis=1)

In [81]:
# convert to datafame and join to original dataset using index column
df_previous_payments_tt = df_previous_payments_tt.to_frame()
df_previous_payments_tt.columns = ['CF']
df_previous_payments_tt_j = df_previous_payments_t.join(df_previous_payments_tt, on=None, how='inner')

In [82]:
df_previous_payments_tt_j.shape

(7, 7)

In [83]:
def get_actual_ear(cf):
    """ derives actual EAR based on all payments based on previous application
    """
    irr = np.irr(cf)
    ear = (1+irr)**360 - 1 # we take 1 year to be 360 days
    return ear

In [84]:
# derive ear based on cashflow column
df_actual_ear = df_previous_payments_tt_j\
.progress_apply(lambda x: get_actual_ear(x.CF), axis=1)

In [85]:
df_final = df_actual_ear.to_frame()
df_final.columns = ['ACTUAL_EAR']
df_final.head(6)

,ACTUAL_EAR
0,0.293995
1,0.524947
2,0.802244
3,0.594947
4,1.360314
5,0.513768


In [86]:
# join results to main dataset using index column
df_final_j = df_previous_payments_tt_j[['SK_ID_CURR','SK_ID_PREV']]\
.join(df_final, on=None, how='inner')

In [87]:
df_final_j

,SK_ID_CURR,SK_ID_PREV,ACTUAL_EAR
0,100042,1125423,0.293995
1,100042,1643959,0.524947
2,100042,1643964,0.802244
3,100042,1716930,0.594947
4,100042,2137382,1.360314
5,100042,2535181,0.513768
6,100042,2604725,0.287169


In [88]:
# aggregate ear at current application level
df_final_j = df_final_j.groupby('SK_ID_CURR')['ACTUAL_EAR']\
.agg(['mean','median','std'])\
.rename(columns={'mean':'MEAN_ACTUAL_EAR','median':'MEDIAN_ACTUAL_EAR', 'std':'STD_ACTUAL_EAR'})

In [89]:
df_final_j

,MEAN_ACTUAL_EAR,MEDIAN_ACTUAL_EAR,STD_ACTUAL_EAR
SK_ID_CURR,,,
100042,0.625341,0.524947,0.369478


In [90]:
# joining expected and actual ear
df_expected_actual = df_final_j.join(df_expected_ear, on='SK_ID_CURR', how='inner')

In [91]:
df_expected_actual

,MEAN_ACTUAL_EAR,MEDIAN_ACTUAL_EAR,STD_ACTUAL_EAR,MEAN_EXPECTED_EAR,MEDIAN_EXPECTED_EAR,STD_EXPECTED_EAR
SK_ID_CURR,,,,,,
100042,0.625341,0.524947,0.369478,0.404926,0.413872,0.222628


In [92]:
# writing only expected ear
expected_ear_csv = '/ws_nparigi/01_Work/04_mba/Year2/Block1/valuation/datascience-assignment/home-credit-default-risk/expected_ear.csv'
#df_expected_ear.to_csv(expected_ear_csv, index=True)
